In [1]:
!python -V

Python 3.9.12


In [1]:
import pandas as pd

In [2]:
import pickle

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [3]:
import mlflow
mlflow.get_tracking_uri()

'sqlite:///mlflow.db'

In [1]:
import mlflow

# define backend (sqlite)
mlflow.set_tracking_uri("sqlite:///mlflow.db")
## define the name of the experiment
mlflow.set_experiment("nyc-taxi-experiment")

In [12]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [18]:
df_train = read_dataframe('/home/yassinfahmy/mlops-training/data/green_tripdata_2024-01.parquet')
df_val = read_dataframe('/home/yassinfahmy/mlops-training/data/green_tripdata_2023-01.parquet')

In [19]:
len(df_train), len(df_val)

(54373, 65946)

In [20]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [21]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [22]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [23]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

6.1661951125966965

In [24]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [25]:
with mlflow.start_run():
    mlflow.set_tag("Developer", "Yassin")
    mlflow.log_param('train_data_path', '/home/yassinfahmy/mlops-training/data/green_tripdata_2024-01.parquet')
    mlflow.log_param('val_data_path', '/home/yassinfahmy/mlops-training/data/green_tripdata_2023-01.parquet')
    alpha = 0.1
    mlflow.log_param('alpha', alpha)

    #build model
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    #test model
    y_pred = lr.predict(X_val)
    #calculate RMSE
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric('rmse', rmse)

    ##save model in mlflow
    mlflow.log_artifact(local_path='./models/lin_reg.bin', artifact_path="Pickled_Model")

# A more complicated example

In [27]:
import xgboost as xgb

In [28]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [29]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [30]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [31]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

#pass the objective model func to fmin
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[0]	validation-rmse:7.69953                                                                                             
[1]	validation-rmse:6.82941                                                                                             
  0%|                                                                            | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/yassinfahmy/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [16:32:14] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[2]	validation-rmse:6.27933                                                                                             
[3]	validation-rmse:5.93497                                                                                             
[4]	validation-rmse:5.72326                                                                                             
[5]	validation-rmse:5.59830                                                                                             
[6]	validation-rmse:5.52042                                                                                             
[7]	validation-rmse:5.47330                                                                                             
[8]	validation-rmse:5.44640                                                                                             
[9]	validation-rmse:5.42587                                                                                             
[10]	validation-rmse:5.41726    

/home/yassinfahmy/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [16:33:00] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[1]	validation-rmse:7.79750                                                                                             
[2]	validation-rmse:7.33280                                                                                             
[3]	validation-rmse:6.95020                                                                                             
[4]	validation-rmse:6.63754                                                                                             
[5]	validation-rmse:6.38276                                                                                             
[6]	validation-rmse:6.17710                                                                                             
[7]	validation-rmse:6.01182                                                                                             
[8]	validation-rmse:5.88065                                                                                             
[9]	validation-rmse:5.77645     

...

/home/yassinfahmy/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [16:48:42] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.85228                                                                                             
[1]	validation-rmse:7.00842                                                                                             
[2]	validation-rmse:6.41903                                                                                             
[3]	validation-rmse:6.01458                                                                                             
[4]	validation-rmse:5.74462                                                                                             
[5]	validation-rmse:5.56951                                                                                             
[6]	validation-rmse:5.45557                                                                                             
[7]	validation-rmse:5.38336                                                                                             
[8]	validation-rmse:5.33560     

/home/yassinfahmy/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [16:49:27] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[1]	validation-rmse:8.30600                                                                                             
[2]	validation-rmse:7.99623                                                                                             
[3]	validation-rmse:7.71768                                                                                             
[4]	validation-rmse:7.46551                                                                                             
[5]	validation-rmse:7.23850                                                                                             
[6]	validation-rmse:7.03531                                                                                             
[7]	validation-rmse:6.85338                                                                                             
[8]	validation-rmse:6.69087                                                                                             
[9]	validation-rmse:6.54483     

KeyboardInterrupt: 

In [37]:
## auto log params
mlflow.xgboost.autolog(disable=True)

In [42]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:squarederror',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
    ## the preferred way to log a model, it logs all dependancies and allows u to load model from mlflow
    mlflow.xgboost.log_model(booster, artifact_path="mlflow_models")

[0]	validation-rmse:8.46333
[1]	validation-rmse:7.97654
[2]	validation-rmse:7.55917
[3]	validation-rmse:7.19736
[4]	validation-rmse:6.88892
[5]	validation-rmse:6.62770
[6]	validation-rmse:6.40627
[7]	validation-rmse:6.21908
[8]	validation-rmse:6.06071
[9]	validation-rmse:5.92925
[10]	validation-rmse:5.82124
[11]	validation-rmse:5.72704
[12]	validation-rmse:5.64961
[13]	validation-rmse:5.58720
[14]	validation-rmse:5.53438
[15]	validation-rmse:5.48941
[16]	validation-rmse:5.45231
[17]	validation-rmse:5.42017
[18]	validation-rmse:5.39411
[19]	validation-rmse:5.37018
[20]	validation-rmse:5.35297
[21]	validation-rmse:5.33415
[22]	validation-rmse:5.31993
[23]	validation-rmse:5.30616
[24]	validation-rmse:5.29627
[25]	validation-rmse:5.28715
[26]	validation-rmse:5.27953
[27]	validation-rmse:5.27321
[28]	validation-rmse:5.26629
[29]	validation-rmse:5.26145
[30]	validation-rmse:5.25634
[31]	validation-rmse:5.25246
[32]	validation-rmse:5.25106
[33]	validation-rmse:5.24831
[34]	validation-rmse:5.2

/home/yassinfahmy/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [17:19:50] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


# load model as python function from mlflow

In [43]:
#load saved model
logged_model = 'runs:/31a2f5df2be54d39a92c41f61a7efabf/mlflow_models'

In [44]:
# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)


# load  model with xgboost 

In [45]:
xgboost_model = mlflow.xgboost.load_model(logged_model)

# try a few different models

In [49]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        # mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

2024/04/08 17:48:10 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/04/08 17:53:53 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/04/08 17:53:58 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/04/08 18:05:32 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
/home/yassinfahmy/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
